In [1]:
import ifcopenshell.util
import ifcopenshell.geom as geom
from ifcopenshell.util.selector import Selector
import open3d as o3d
import numpy as np
import time
import os
import trainingdata

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def mergeclass_subclouds(class_config):
    for Class in class_config:
        classpcd = o3d.geometry.PointCloud()
        if len(Class[3]) > 1:
            id = 0
            while id < len(Class[3]):
                pcd = Class[3][id]
                classpcd.__iadd__(pcd)
                id = id + 1
            Class[3] = classpcd
        else:
            Class[3] = Class[3][0]


In [3]:
def loop_projects(path, classes):
    directory_contents = os.listdir(path)
    for item in directory_contents:
        if os.path.isdir(os.path.join(path,item)) and "-BIM-" in item:
            (projectname, ifc_folder, pointcloud_folder) = trainingdata.load_project(os.path.join(path,item))
            count=0
            class_config = []
            for Class in classes:
                class_config.append([Class[0],Class[1],count,[]])
                count = count+1
            for Class in class_config:
                classrefcloud = trainingdata.IFCtoO3d(ifc_folder, Class[1], voxel_size = 0.01)
                Class[3].append(classrefcloud)
            trainingdata.mergeclass_subclouds(class_config)
            reference_pointcloud = trainingdata.generate_refpcd(class_config) 
            labeled_pointcloud = trainingdata.generate_trainingdata(reference_pointcloud, pointcloud_folder)
            class_pointclouds = trainingdata.Split_trainingsdata(labeled_pointcloud , class_config, clutter = True)
            trainingdata.Save_trainingsdata(class_pointclouds, path, name = projectname)
   

In [5]:
class_config = [("Wall",["IfcWall","IfcDoor","IfcWindow"]),("Floor",["IfcSlab"])]
loop_projects(r"D:\SAM-Repo\scan-to-bim-python\Samples\Projects", class_config)